# Tugas 3 (TimeSeries Dataset airline-passengers)

## Get Dataset

In [1]:
import pandas as pd
# transform univariate time series to supervised learning problem
from numpy import array

In [2]:
# load time series dataset
series = pd.read_csv('https://raw.githubusercontent.com/dhamvi01/Univariate-Time-Series-using-LSTM/master/airline-passengers.csv', header=0, index_col=0)
# series = series.values.

In [3]:
df = series['Passengers']

In [4]:

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)
# define univariate time series

print(df.shape)
# transform to a supervised learning problem
X, y = split_sequence(df, 3)
print(X.shape, y.shape)
# show each sample
for i in range(len(X)):
  print(X[i], y[i])

(144,)
(141, 3) (141,)
[112 118 132] 129
[118 132 129] 121
[132 129 121] 135
[129 121 135] 148
[121 135 148] 148
[135 148 148] 136
[148 148 136] 119
[148 136 119] 104
[136 119 104] 118
[119 104 118] 115
[104 118 115] 126
[118 115 126] 141
[115 126 141] 135
[126 141 135] 125
[141 135 125] 149
[135 125 149] 170
[125 149 170] 170
[149 170 170] 158
[170 170 158] 133
[170 158 133] 114
[158 133 114] 140
[133 114 140] 145
[114 140 145] 150
[140 145 150] 178
[145 150 178] 163
[150 178 163] 172
[178 163 172] 178
[163 172 178] 199
[172 178 199] 199
[178 199 199] 184
[199 199 184] 162
[199 184 162] 146
[184 162 146] 166
[162 146 166] 171
[146 166 171] 180
[166 171 180] 193
[171 180 193] 181
[180 193 181] 183
[193 181 183] 218
[181 183 218] 230
[183 218 230] 242
[218 230 242] 209
[230 242 209] 191
[242 209 191] 172
[209 191 172] 194
[191 172 194] 196
[172 194 196] 196
[194 196 196] 236
[196 196 236] 235
[196 236 235] 229
[236 235 229] 243
[235 229 243] 264
[229 243 264] 272
[243 264 272] 237
[264 

In [5]:
newX = pd.DataFrame(X,columns=['Xt-2','Xt-1','Xt'])
newY = pd.DataFrame(y, columns=['Output'])

In [6]:
new_data_passenger = pd.concat([newX,newY],axis=1)
new_data_passenger  

,Xt-2,Xt-1,Xt,Output
0,112,118,132,129
1,118,132,129,121
2,132,129,121,135
3,129,121,135,148
4,121,135,148,148
...,...,...,...,...
136,472,535,622,606
137,535,622,606,508
138,622,606,508,461
139,606,508,461,390


## PrePosessing Data

### Drop Output 

In [7]:
new_data_passenger['Output'] =y

In [8]:
X = new_data_passenger.drop(columns=['Output'])

### Proses Minmax

In [9]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
scaled = scaler.fit_transform(X)
features_names = X.columns.copy()
scaled_features = pd.DataFrame(scaled, columns=features_names)
scaled_features.head(10)


,Xt-2,Xt-1,Xt
0,0.015444,0.027027,0.054054
1,0.027027,0.054054,0.048263
2,0.054054,0.048263,0.032819
3,0.048263,0.032819,0.059846
4,0.032819,0.059846,0.084942
5,0.059846,0.084942,0.084942
6,0.084942,0.084942,0.061776
7,0.084942,0.061776,0.028958
8,0.061776,0.028958,0.000000
9,0.028958,0.000000,0.027027


### Get Min and Max Value From Xt

In [10]:
max_xt = scaled_features['Xt'].max()
min_xt = scaled_features['Xt'].min()
print(max_xt,"|" , min_xt)

1.0 | 0.0


### Split Data

In [11]:
X_train, X_test = train_test_split(scaled_features, train_size = 0.8, test_size = 0.2, shuffle=False)
y_train, y_test = train_test_split(y, train_size = 0.8, test_size = 0.2, shuffle=False)

## Implementasi Ke Model

### KNN (n=3)

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [13]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)

Get error and matrix 

In [14]:
y_test

array([505, 404, 359, 310, 337, 360, 342, 406, 396, 420, 472, 548, 559,
       463, 407, 362, 405, 417, 391, 419, 461, 472, 535, 622, 606, 508,
       461, 390, 432])

In [15]:
y_pred

array([355, 347, 305, 305, 271, 340, 318, 312, 435, 355, 306, 355, 347,
       347, 305, 305, 305, 355, 355, 305, 306, 355, 347, 347, 347, 347,
       305, 305, 305])

In [16]:
print(y_test.shape ,"|",y_pred.shape)

(29,) | (29,)


In [17]:
MSE = mean_squared_error(y_test, y_pred) # Calculating MSE 
print(MSE)

17343.896551724138
